# Creating a loader with Kosh

In [1]:
import os
import kosh
from  kosh import KoshLoader

# Make sure local file is new sql file
kosh_example_sql_file = "kosh_ultra_example.sql"
    
# Create db on file
store = kosh.create_new_db(kosh_example_sql_file)

In [2]:
# Add MNST datasets to store
sample = store.create(name="example", metadata={'project':"example"})


In [3]:
# Associate files with datasets
sample.associate("example.ultra", mime_type="my_ultra")

'3e0555ec0be8426880610bf5be587f47'

In [4]:
import numpy

class MyUltraLoader(KoshLoader):
    types ={"my_ultra": ["numpy", ]}
    def load_from_ultra(self, variable):
        if not isinstance(variable, (list,tuple)):  # only one variable requested
            variable = [variable,]
        variables = [[],] * len(variable)
        previous_line = ""
        var_names = None
        with open(self.obj.uri, "rb") as f:
            for line in f.readlines():
                # Skip headers
                line = line.decode("utf-8")
                if line[0]=="#" or line.strip()=="end":
                    previous_line = line
                    var_names = None
                    continue
                if var_names is None:
                    var_names = previous_line.split()[1:]
                    # clean up name list
                    while "vs" in var_names:
                        var_names.remove("vs")
                sp = line.split()
                for ivar, name in enumerate(variable):
                    if name in var_names:
                        index = var_names.index(name)
                        variables[ivar].append(float(sp[index]))
        # we're done reading these variables, co
        for ivar in range(len(variables)):
            if len(variables[ivar]) > 0 and isinstance(variable[ivar], list):
                variables[ivar] = numpy.array(variables[ivar])
        if len(variables) > 1:
            return variables
        else:  # only one variable read in
            return variables[0]
        
    def __init__(self, obj):
        super(MyUltraLoader, self).__init__(obj)

    def extract(self, *args, **kargs):
        return self.load_from_ultra(self.feature)
        
    def list_features(self):
        variables = set()
        with open(self.obj.uri, "r") as f:
            previous = ""
            for line in f.readlines():
                if line[0] == "#":
                    previous = line
                    var_names = None
                    continue
                if var_names is not None:
                    continue
                var_names = previous.split()[1:]
                while "vs" in var_names:
                    var_names.remove("vs")
                for name in var_names:
                    variables.add(name)
        return list(variables)    

store.add_loader(MyUltraLoader)

In [5]:
print(sample.list_features())

['var2', 'energy', 'time']


In [6]:
print(sample.get("energy"))

[0.6, 0.7, 0.8, 0.6, 0.5, 0.2, 0.1, 0.6]
